In [1]:
!pip install ohmeow-blurr -q
!pip install bert-score -q

In [2]:
# !pip install pytest

In [3]:
import pandas as pd
from fastai.text.all import *
from transformers import *
from blurr.data.all import *
from blurr.modeling.all import *

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [4]:
df = pd.read_csv('./sosum-data/processed_combined_data.csv', sep=',')
df = df.dropna().reset_index()

In [5]:
#Clean text
df['content'] = df['content'].apply(lambda x: x.replace('/',''))
df['content'] = df['content'].apply(lambda x: x.replace('\xa0',''))
df.head()

,index,content,title
0,0,"<em>Note, this answer is for Python 2.x as it was written in 2008, metaclasses are slightly different in 3.x, see the comments.<em> Metaclasses are the secret sauce that make 'class' work. The default metaclass for a new style object is called 'type'. Metaclasses take 3 args. '<strong>name<strong>', '<strong>bases<strong>' and '<strong>dict<strong>' Here is where the secret starts. Look for where name, bases and the dict come from in this example class definition. Lets define a metaclass that will demonstrate how '<strong>class:<strong>' calls it. And now, an example that actually means so...","Metaclasses are the secret sauce that make 'class' work.The default metaclass for a new style object is called 'type'.Metaclasses take 3 args.'<strong>name</strong>', '<strong>bases</strong>' and '<strong>dict</strong>'"
1,1,"I think the ONLamp introduction to metaclass programming is well written and gives a really good introduction to the topic despite being several years old already. http:www.onlamp.compubapython20030417metaclasses.html (archived at https:web.archive.orgweb20080206005253http:www.onlamp.compubapython20030417metaclasses.html) In short: A class is a blueprint for the creation of an instance, a metaclass is a blueprint for the creation of a class. It can be easily seen that in Python classes need to be first-class objects too to enable this behavior. I've never written one myself, but I think on...","In short: A class is a blueprint for the creation of an instance, a metaclass is a blueprint for the creation of a class."
2,2,"One use for metaclasses is adding new properties and methods to an instance automatically. For example, if you look at Django models, their definition looks a bit confusing. It looks as if you are only defining class properties: However, at runtime the Person objects are filled with all sorts of useful methods. See the source for some amazing metaclassery.",One use for metaclasses is adding new properties and methods to an instance automatically.
3,3,"A metaclass is the class of a class. Like a class defines how an instance of the class behaves, a metaclass defines how a class behaves. A class is an instance of a metaclass. While in Python you can use arbitrary callables for metaclasses (like Jerub shows), the more useful approach is actually to make it an actual class itself. <code>type<code> is the usual metaclass in Python. In case you're wondering, yes, <code>type<code> is itself a class, and it is its own type. You won't be able to recreate something like <code>type<code> purely in Python, but Python cheats a little. To create your...","A metaclass is the class of a class.Like a class defines how an instance of the class behaves, a metaclass defines how a class behaves.A class is an instance of a metaclass.To create your own metaclass in Python you really just want to subclass <code>type</code>.The metaclass is determined by looking at the baseclasses of the class-to-be (metaclasses are inherited), at the <code>__metaclass__</code> attribute of the class-to-be (if any) or the <code>__metaclass__</code> global variable."
4,4,"<strong>Main messages :)<strong> De-cyphered: The Zen of Python, by Tim Peters Beautiful is better than ugly. Explicit is better than implicit. Simple is better than complex. Complex is better than complicated. Flat is better than nested. Sparse is better than dense. Readability counts. Special cases aren't special enough to break the rules. Although practicality beats purity. Errors should never pass silently. Unless explicitly silenced. In the face of ambiguity, refuse the temptation to guess. There should be one-- and preferably only one --obvious way to do it. Although that way may not...","The Zen of Python, by Tim Peters"


In [6]:
#Select only part of it (makes testing faster)
articles = df

In [7]:
#Truncate text to make it fit into the model
#articles['content'] = articles['content'].apply(lambda x: ' '.join(x.split()[:700]))

## Import model and set up data

In [8]:
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, 
                                                                  model_cls=BartForConditionalGeneration)

In [9]:
#text_gen_kwargs = default_text_gen_kwargs(hf_config, hf_model, task='summarization');

In [10]:
hf_batch_tfm = HF_Seq2SeqBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model, task='summarization',
text_gen_kwargs={'max_length': 248,
 'min_length': 56,
 'do_sample': False,
 'early_stopping': True,
 'num_beams': 4,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'repetition_penalty': 1.0,
 'bad_words_ids': None,
 'bos_token_id': 0,
 'pad_token_id': 1,
 'eos_token_id': 2,
 'length_penalty': 2.0,
 'no_repeat_ngram_size': 3,
 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1,
 'decoder_start_token_id': 2,
 'use_cache': True,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'output_attentions': False,
 'output_hidden_states': False,
 'output_scores': False,
 'return_dict_in_generate': False,
 'forced_bos_token_id': 0,
 'forced_eos_token_id': 2,
 'remove_invalid_values': False})

blocks = (HF_Seq2SeqBlock(before_batch_tfm=hf_batch_tfm), noop)

dblock = DataBlock(blocks=blocks, get_x=ColReader('content'), get_y=ColReader('title'), splitter=RandomSplitter())

In [11]:
dls = dblock.dataloaders(articles, bs=2)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


### Training


In [12]:
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]
        }
    }

In [13]:
model = HF_BaseModelWrapper(hf_model)
learn_cbs = [HF_BaseModelCallback]
fit_cbs = [HF_Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics),  Recorder()]

learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(seq2seq_splitter, arch=hf_arch)).to_fp16()

learn.create_opt() 
learn.freeze()

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\blurr\modeling\seq2seq\core.py:42: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  compute_func = hf_load_metric(metric_name).compute


In [14]:
learn.cbs

(#6) [TrainEvalCallback,Recorder,CastToTensor,ProgressCallback,HF_BaseModelCallback,MixedPrecision]

In [15]:
learn.fit_one_cycle(8, lr_max=3e-5, cbs=fit_cbs)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\amp\autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\cuda\amp\grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1,time
0,0.359727,0.393375,0.472308,0.423875,0.452445,0.727543,0.860782,0.784149,1:30:33
0,0.359727,0.393375,0.472308,0.423875,0.452445,0.727543,0.860782,0.784149,1:30:33
1,0.315535,0.481088,0.461842,0.413766,0.443018,0.725496,0.852945,0.779969,1:27:03
1,0.315535,0.481088,0.461842,0.413766,0.443018,0.725496,0.852945,0.779969,1:27:03
2,0.241778,0.308950,0.481039,0.429664,0.457343,0.741023,0.859881,0.792666,1:16:42
2,0.241778,0.308950,0.481039,0.429664,0.457343,0.741023,0.859881,0.792666,1:16:42
3,0.193641,0.360760,0.495954,0.450137,0.478615,0.734213,0.868658,0.791672,1:24:50
3,0.193641,0.360760,0.495954,0.450137,0.478615,0.734213,0.868658,0.791672,1:24:50
4,0.159992,0.444575,0.487907,0.438288,0.466458,0.733088,0.866806,0.790678,1:22:29
4,0.159992,0.444575,0.487907,0.438288,0.466458,0.733088,0.866806,0.790678,1:22:29


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


### Generate predictions

In [16]:
df['content'][0]

'<em>Note, this answer is for Python 2.x as it was written in 2008, metaclasses are slightly different in 3.x, see the comments.<em> Metaclasses are the secret sauce that make \'class\' work. The default metaclass for a new style object is called \'type\'. Metaclasses take 3 args. \'<strong>name<strong>\', \'<strong>bases<strong>\' and \'<strong>dict<strong>\' Here is where the secret starts. Look for where name, bases and the dict come from in this example class definition. Lets define a metaclass that will demonstrate how \'<strong>class:<strong>\' calls it. And now, an example that actually means something, this will automatically make the variables in the list "attributes" set on the class, and set to None. Note that the magic behaviour that \'Initalised\' gains by having the metaclass <code>init_attributes<code> is not passed onto a subclass of Initalised. Here is an even more concrete example, showing how you can subclass \'type\' to make a metaclass that performs an action when 

In [17]:
outputs = learn.blurr_generate(df['content'][0], early_stopping=False, num_return_sequences=1)

for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (142) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


=== Prediction 1 ===
 Metaclasses are the secret sauce that make 'class' work.The default metaclass for a new style object is called 'type'.Metaclasses take 3 args.'<strong>name</strong>', '</strong><strong>bases</strong></strong>' and '<strong><dict</strong>(Here is where the secret starts.Look for where name, bases and the dict come from in this example class definition.Now, an example that actually means something, this will automatically make the variables in the list "attributes" set on the class, and set to None.<em>Note, this answer is for Python 2.x as it



In [18]:
# save this model
learn.save('6epochs-bart_sosum_finetuned-opt')
learn.save('6epochs-bart_sosum_finetuned', with_opt=False)

Path('models/6epochs-bart_sosum_finetuned.pth')

In [19]:
learn.export('6epochs-bart_sosum_finetuned.pkl')

TypeError: cannot pickle '_thread.lock' object